# Иерархическая категоризация продуктов в e-commerce

## Формулировка задачи

У каждого товара есть:

- *id* - идентификатор товара
- *title - заголовок*
- *short_description - краткое описание*
- *name_value_characteristics - название:значение* характеристики товара, может быть несколько для одного товара и для одной характеристик. Пример: `name1: value1 | value2 | valueN_1 / name2: value1 | value2 | valueN_2 / nameK: value1 | value2 | valueN_K`
- *rating - средний рейтинг товара*
- *feedback_quantity - количество отзывов по товару*
- *category_id - категория товара(таргет)*

Нужно предсказать категории (category_id) для переменных из файла test.parquet, используя для тренировки данные из train.parquet.

## Решение

In [1]:
# Базовые модули
import pandas as pd
import numpy as np
import re

# NLP-модули
import nltk
import pymorphy2
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# Моделирование
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.svm import LinearSVC

Подгрузим тренировочные и тестовые данные.

In [2]:
df_train = pd.read_parquet('ke_test_data/train.parquet')
df_test = pd.read_parquet('ke_test_data/test.parquet')
df_train.head(3)

,id,title,short_description,name_value_characteristics,rating,feedback_quantity,category_id
0,1267423,Muhle Manikure Песочные колпачки для педикюра ...,Muhle Manikure Колпачок песочный шлифовальный ...,None,0.000000,0,2693
1,128833,"Sony Xperia L1 Защитное стекло 2,5D",None,None,4.666667,9,13408
2,569924,"Конверт для денег Прекрасная роза, 16,5 х 8 см","Конверт для денег «Прекрасная роза», 16,5 × 8 см",None,5.000000,6,11790


Видим, что в части данных много Null-ов.

In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283452 entries, 0 to 283451
Data columns (total 7 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id                          283452 non-null  int64  
 1   title                       283452 non-null  object 
 2   short_description           133130 non-null  object 
 3   name_value_characteristics  50360 non-null   object 
 4   rating                      283452 non-null  float64
 5   feedback_quantity           283452 non-null  int64  
 6   category_id                 283452 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 15.1+ MB


In [4]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70864 entries, 0 to 70863
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          70864 non-null  int64  
 1   title                       70864 non-null  object 
 2   short_description           33346 non-null  object 
 3   name_value_characteristics  12576 non-null  object 
 4   rating                      70864 non-null  float64
 5   feedback_quantity           70864 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 3.2+ MB


Подгрузим данные из файла категорий.

In [5]:
df_categories = pd.read_csv('ke_test_data/categories_tree.csv')
print(f'Всего строк: {df_categories.shape[0]}')
df_categories[df_categories['parent_id'] == 0]

Всего строк: 3370


,id,title,parent_id
0,1,Все категории,0
21,553,Травяные сборы,0
23,1754,Табак,0
28,2000,Игры и софт,0


Создаем столбцы уровней иерархии (двигаемся от вершины древа категорий (parent_id = 0)). Всего находим 6 уровней.

In [6]:
# создаем датафреймы под категорию каждой степени детализации
cat_lvl1 = df_categories[df_categories['parent_id'] == 0][['id', 'parent_id']]
cat_lvl1.columns = ['lvl1', 'lvl0']
cat_lvl2 = df_categories[df_categories['parent_id'].isin(cat_lvl1['lvl1'])][['id', 'parent_id']]
cat_lvl2.columns = ['lvl2', 'lvl1']
cat_lvl3 = df_categories[df_categories['parent_id'].isin(cat_lvl2['lvl2'])][['id', 'parent_id']]
cat_lvl3.columns = ['lvl3', 'lvl2']
cat_lvl4 = df_categories[df_categories['parent_id'].isin(cat_lvl3['lvl3'])][['id', 'parent_id']]
cat_lvl4.columns = ['lvl4', 'lvl3']
cat_lvl5 = df_categories[df_categories['parent_id'].isin(cat_lvl4['lvl4'])][['id', 'parent_id']]
cat_lvl5.columns = ['lvl5', 'lvl4']
cat_lvl6 = df_categories[df_categories['parent_id'].isin(cat_lvl5['lvl5'])][['id', 'parent_id']]
cat_lvl6.columns = ['lvl6', 'lvl5']

# проверка не то, что ничего не потеряли
# 20 недостающих категорий - лишние, они все равно не используются в тренировочном сете
print([len(x) for x in [cat_lvl1, cat_lvl2,cat_lvl3,cat_lvl4,cat_lvl5,cat_lvl6]])
print(sum([len(x) for x in [cat_lvl1, cat_lvl2,cat_lvl3,cat_lvl4,cat_lvl5,cat_lvl6]]))

[4, 19, 181, 1392, 1691, 63]
3350


Собираем путь ко всем представленным категориям, начиная с самых детальных.

In [7]:
cat_merged = cat_lvl6.merge(cat_lvl5, how = 'outer', left_on = 'lvl5', right_on = 'lvl5')
cat_merged = cat_merged.merge(cat_lvl4, how = 'outer', left_on = 'lvl4', right_on = 'lvl4')
cat_merged = cat_merged.merge(cat_lvl3, how = 'outer', left_on = 'lvl3', right_on = 'lvl3')
cat_merged = cat_merged.merge(cat_lvl2, how = 'outer', left_on = 'lvl2', right_on = 'lvl2')
cat_merged = cat_merged.merge(cat_lvl1, how = 'outer', left_on = 'lvl1', right_on = 'lvl1')
cat_merged.head(3)

,lvl6,lvl5,lvl4,lvl3,lvl2,lvl1,lvl0
0,2824.0,2812.0,2808.0,2807.0,10014.0,1,0
1,2825.0,2812.0,2808.0,2807.0,10014.0,1,0
2,2826.0,2812.0,2808.0,2807.0,10014.0,1,0


Проставляем длину ветки древа и id конечного листа для ветки древа.

In [8]:
cat_merged = cat_merged.astype('Int64')
cat_merged['deep_id'] = cat_merged.fillna(method='bfill', axis=1).iloc[:, 0]
cat_merged['deep_lvl'] = cat_merged.apply(pd.Series.first_valid_index, axis=1)
cat_merged.head(3)

,lvl6,lvl5,lvl4,lvl3,lvl2,lvl1,lvl0,deep_id,deep_lvl
0,2824,2812,2808,2807,10014,1,0,2824,lvl6
1,2825,2812,2808,2807,10014,1,0,2825,lvl6
2,2826,2812,2808,2807,10014,1,0,2826,lvl6


Как видим ниже, мы покрыли все id в тренировочном сете.

In [9]:
print('Упущенные категории: ', len(df_train[df_train['category_id'].isin(cat_merged['deep_id'].values) == False]))

Упущенные категории:  0


Присоединим id к тренировочному сету.

In [10]:
df_train = df_train.merge(cat_merged, how='left', left_on='category_id', right_on='deep_id')
df_train.head(3)

,id,title,short_description,name_value_characteristics,rating,feedback_quantity,category_id,lvl6,lvl5,lvl4,lvl3,lvl2,lvl1,lvl0,deep_id,deep_lvl
0,1267423,Muhle Manikure Песочные колпачки для педикюра ...,Muhle Manikure Колпачок песочный шлифовальный ...,None,0.000000,0,2693,<NA>,2693,10355,10113,10012,1,0,2693,lvl5
1,128833,"Sony Xperia L1 Защитное стекло 2,5D",None,None,4.666667,9,13408,<NA>,13408,10398,10044,10020,1,0,13408,lvl5
2,569924,"Конверт для денег Прекрасная роза, 16,5 х 8 см","Конверт для денег «Прекрасная роза», 16,5 × 8 см",None,5.000000,6,11790,<NA>,11790,11163,10118,10018,1,0,11790,lvl5


Объединив древо категорий с тренировочным фреймом, можем приступать к препроцессингу тренировочного фрейма.

## Препроцессинг текста

Подгружаем нужные данные из модулей для NLP.

In [11]:
morph_pymorphy2 = pymorphy2.MorphAnalyzer()
morph_nltk = WordNetLemmatizer()
nltk.download('omw-1.4')
nltk.download('wordnet')
stopwords_nltk = nltk.corpus.stopwords.words(['russian', 'english'])

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Keklek\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Keklek\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Пишем функции для препроцессинга ячеек.

In [12]:
def get_wordnet_pos(word):
    
    """ Определение части речи для лемматизации английских слов от NLTK """
    
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

# выносим паттерны за функции
cyrillic_pat = re.compile(r'[А-Яа-я]')
english_pat = re.compile(r'[A-Za-z]')
pat_multiply = re.compile(r'(\d)[XxХх×*](\d)')
pat_multiply_2 = re.compile(r'(\d)\s[XxХх×*]\s(\d)')
pat_dot = re.compile(r'(\d)[,.](\d)')  
punctuation = re.compile('[”“«»!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~.-]')

def preproc_text(text, stopwords=stopwords_nltk, cyrillic_pat=cyrillic_pat, english_pat=english_pat,
                pat_multiply=pat_multiply, pat_multiply_2=pat_multiply_2, pat_dot=pat_dot, punctuation=punctuation):
    
    ''' Основная функция препроцессинга - удаляет пунктуацию, приводит размеры и нецелые числа к единому формату, 
    удаляет лишние пробелы, делает лемматизацию
    '''

    # приведение паттернов в единообразный вид и удаление лишней пунктуации
    text = re.sub(pat_multiply,'\g<1>mlt\g<2>', text)
    text = re.sub(pat_multiply_2,'\g<1>mlt\g<2>', text)
    text = re.sub(pat_dot, '\g<1>dot\g<2>', text)
    text = re.sub(punctuation, ' ', text)
    
    # лемматизация
    words = [word for word in text.split() if word not in (stopwords)]                     
    res =[]
    for word in words:

        if len(word) <= 3:
            # pymorphy часто неправильно воспринимает короткие слова 
            # как часть длинного слова-омонима (напр., см = смотри...)
            # сделаем под это исключение
            res.append(word)
        elif bool(re.search(cyrillic_pat, word)):
            p = morph_pymorphy2.parse(word)[0]
            res.append(p.normal_form)
        elif bool(re.search(english_pat, word)):
            p = morph_nltk.lemmatize(word, get_wordnet_pos(word))
            res.append(p)
    return(' '.join(res)) 

Проверка, что все ок.

In [13]:
check = '   Магнит        символ Нового 3,3 3 x3 3 X 3 3*3 года-Тигренок/(по 3 шт в уп better using in with washed)'
preproc_text(check)

'магнит символ новый 3dot3 3 x3 3mlt3 3mlt3 год тигрёнок 3 шт уп well use wash'

Функция препроцессинга текстовых колонок датафрейма.

In [14]:
def preproc_df(df):
    
    ''' Делает препроцессинг колонок с текстом, используя функцию preproc_text '''
    
    df.replace(to_replace='None', value='', ) # Избавляемся от None-ов
    df.fillna(0, inplace=True) # Избавляемся от None-ов
    df[['title', 'short_description', 'name_value_characteristics']] = df[['title', 
                                                                           'short_description', 
                                                                           'name_value_characteristics']].astype(str)    
    df['title'] = df[['title', 'short_description', 'name_value_characteristics']].apply(
        lambda x: ','.join(x.dropna().astype(str)), axis=1)
    print('>>> Merged columns')
    df['title'] = df['title'].str.lower()        
    df['title'] = df['title'].apply(lambda x: preproc_text(x))
    print('>>> Done with title')
    print('>>> Done with name_value_characteristics')

In [15]:
%%time
preproc_df(df_train)
df_train.head(3)

>>> Merged columns
>>> Done with title
>>> Done with name_value_characteristics
Wall time: 10min 24s


,id,title,short_description,name_value_characteristics,rating,feedback_quantity,category_id,lvl6,lvl5,lvl4,lvl3,lvl2,lvl1,lvl0,deep_id,deep_lvl
0,1267423,muhle manikure песочный колпачок педикюр pw ср...,Muhle Manikure Колпачок песочный шлифовальный ...,0,0.000000,0,2693,0,2693,10355,10113,10012,1,0,2693,lvl5
1,128833,sony xperia l1 защитный стекло 2dot5d 0dot0,0,0,4.666667,9,13408,0,13408,10398,10044,10020,1,0,13408,lvl5
2,569924,конверт деньга прекрасный роза 16dot5mlt8 см к...,"Конверт для денег «Прекрасная роза», 16,5 × 8 см",0,5.000000,6,11790,0,11790,11163,10118,10018,1,0,11790,lvl5


## Моделирование

В текущей задаче иерархической классификации нам крайне важно спускаться к листьям по правильным "веткам". Было решено сделать классификацию сверху-вниз, остановкой классификации будет достижения листа "ветки".

Видим, что со 2-го уровня иерархии начинается деление на классы. Причем классы местами очень несбалансированные.

In [16]:
print('lvl1')
print(df_train[df_train['lvl1'] != 'none']['lvl1'].value_counts(normalize=True))
print('\nlvl2')
print(df_train[df_train['lvl2'] != 'none']['lvl2'].value_counts(normalize=True))
print('\nlvl3')
print(df_train[df_train['lvl3'] != 'none']['lvl3'].value_counts(normalize=True))
print('\nlvl4')
print(df_train[df_train['lvl4'] != 'none']['lvl4'].value_counts(normalize=True))
print('\nlvl5')
print(df_train[df_train['lvl5'] != 'none']['lvl5'].value_counts(normalize=True))
print('\nlvl6')
print(df_train[df_train['lvl6'] != 'none']['lvl6'].value_counts(normalize=True))

lvl1
1    1.0
Name: lvl1, dtype: Float64

lvl2
10018    0.254421
10014    0.195423
10003     0.19507
10012    0.182867
10020     0.17222
Name: lvl2, dtype: Float64

lvl3
10023    0.135713
10044    0.112696
10116    0.101118
10118     0.08225
10115    0.079079
10091    0.039693
10137     0.03819
10191    0.031818
10113    0.031166
10052    0.030058
10024    0.027952
10221    0.025697
10070    0.025599
10165    0.024953
10021    0.024166
10110    0.021997
10049    0.018373
10073    0.017209
10222    0.016331
10144    0.013311
10095    0.011374
10219    0.009342
10094    0.007941
10074    0.007793
10129    0.006534
2807     0.006297
10184    0.006234
10084     0.00532
10141    0.005045
10163    0.004967
10026    0.004872
10058    0.003521
10180    0.002956
10166    0.002706
10022    0.002367
10030    0.002353
2894     0.001806
10186    0.001778
10232    0.001704
10169    0.001588
10034    0.001309
10226     0.00103
10185    0.000988
10086    0.000762
2673     0.000759
12084    0.000533
10

В условиях ограниченных мощностей и времени, мы сначала проверили RandomForest и логистическую регрессию на базовых настройках. <br> 
RandomForest значительно слабе показал себя на самых верхних уровнях древа классификации, следовательно, было решено отбросить классификаторы, базирующиеся на деревьях решений, а использовать вместо них линейные классификаторы. <br> Преимуществом линейных классификаторов является также очень высокая скорость обучения.

В моделе будем использовать подход bag of words с использованием TFIDF-векторизатора, который занижает веса высокочастотных слов. Будем считать частоты по словам и парам слов (использовать юниграммы и биграммы).

In [17]:
tfidf = TfidfVectorizer(ngram_range=(1,2))
label_encoder = LabelEncoder()

В качестве классификатора выбирали между моделью логистической регрессии и линейным SVM. Второй показал лучшие результаты практически на всех уровнях древа классификации.

Пропишем параметры классификатора.

In [18]:
clf = LinearSVC(penalty='l2', class_weight='balanced', random_state=0, max_iter=100000, tol=1e-5, C=10)

Пропишем также функцию валидации модели. Для низкоуровневых моделей, по которым не провести кросс-валидацию из-за малого числа представителей одного из классов, заменяем кросс-валидационный F1-скор на F1-скор по тестовой выборке.

In [19]:
# содаем механизм скоринга
scoring = make_scorer(f1_score, average = 'macro')

# сохраняем баланс классов при делении на кросс-валидации
stratified_kfold = StratifiedKFold(n_splits=3)

def try_cval(Z_train, y_train, f1_holdout, clf=clf, 
             scoring=scoring, cv=stratified_kfold):
    
    '''
    Пробует провести кросс-валидацию модели с использованием метрики f1,
    если не получается, то использует holdout f1
    '''
    
    try:
        cv_score = cross_val_score(estimator=clf, X=Z_train, y=y_train, 
                        scoring=scoring, cv=cv, n_jobs=-1)
        res = [cv_score.mean(), cv_score.std()]
    except ValueError:
        res = [f1_holdout, 0]
    return(res)

Прогоним модель по всем уровням

In [20]:
%%time
df_predictions = pd.DataFrame()

# задаем листы с предками-потомками, по которым будем итерировать модели
p_c = ['lvl1', 'lvl2', 'lvl3', 'lvl4', 'lvl5']
d_c = ['lvl2', 'lvl3', 'lvl4', 'lvl5', 'lvl6']

res = []

# основной цикл
for p, d in zip(p_c, d_c):
    
    # движемся от начала древа вниз. фильтруем по классу-предку, классифицируем по подклассам на уровень ниже
    for i in df_train[(df_train[p] != 0) & (df_train[d] != 0)][p].unique():
        
        # считаем длину тренировочной выборки, понадобится для расчета средневзвешенных значений коэффициентов
        total_length = df_train[(df_train[p] != 0) & (df_train[d] != 0)][p].shape[0]
        
        # выбираем Х и у с единым предком из тренировочной выборки
        X = df_train[(df_train[p] == i)]['title'].astype(str)
        y = df_train[(df_train[p] == i)][d]
        
        # логгирование - начало итерации
        # print('working on >>> ', i, end = '')
        
        # были категории-родители с только одним потомком - для таких присваиваем руками класс единственного потомка
        if len(y.unique()) == 1:
            y_pred = y
            f1_holdout = 1
            df_predictions_small = pd.DataFrame({'X_test': X_test,                               
                                                 'y_pred': y_pred,                                      
                                                 'y_test': y_pred})
            cval_mean = 1
            cval_std = 0
        else:
            
            # были очень маленькие категории, для которых не сделать сплит на трейн/тест
            # для мелких моделей позволим оверфит
            if len(y)//4 < 2:
                X_train, X_test, y_train, y_test = X, X, y, y
            
            # создание модели и предсказание в стандартных ситуациях
            else:
                # сплит данных на трейн/тест
                X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                                    random_state=0, stratify=y, 
                                                                    test_size=0.25) # Оставляем баланс классов при сплите
            # трансформация переменных
            Z_train = tfidf.fit_transform(X_train)
            Z_test = tfidf.transform(X_test)
            y_train = label_encoder.fit_transform(y_train)
            y_test = label_encoder.transform(y_test)
            
            # фиттинг
            clf.fit(Z_train, y_train)
            y_pred = clf.predict(Z_test)
            
            # расчет метрик качества
            f1_holdout = f1_score(y_test, y_pred, average='macro')
            cval = try_cval(Z_train=Z_train, y_train=y_train, 
                            f1_holdout=f1_holdout, cv=stratified_kfold)
            cval_mean = cval[0]
            cval_std = cval[1]
            cval_mean_weighted = cval_mean*len(X)/total_length
            
            # запись результатов в мини-датафрейм
            df_predictions_small = pd.DataFrame({'X_test': X_test,                                
                                                 'y_pred': label_encoder.inverse_transform(y_pred),                                      
                                                 'y_test': label_encoder.inverse_transform(y_test)})
        
        # присоединение результатов мини-датафрейма к основному датафрейму
        df_predictions = pd.concat([df_predictions, df_predictions_small], axis=0)
        res.append([i, d, p, f1_holdout, len(X), len(X)/total_length, 
                    f1_holdout*len(X)/total_length, cval_mean, cval_std, cval_mean_weighted])    
        df_res = pd.DataFrame(res)
        df_res.columns = ['i', 'predicted_lvl', 'parent_lvl', 'f1_holdout', 'length', 
                          'wgt_in_lvl', 'f1_lvl_weighted', 'cval_f1_mean', 'cval_f1_std', 'cval_mean_weighted']
        
        # логгирование - конец итерации
        print('parent >>>|', p, '|<<< child >>>|', d, '|<<< predicted for parent >>> ', i, ' <<<', end='')
print('>>> DONE WITH WHOLE DF <<<')
df_res

parent >>>| lvl1 |<<< child >>>| lvl2 |<<< predicted for parent >>>  1  <<<parent >>>| lvl2 |<<< child >>>| lvl3 |<<< predicted for parent >>>  10012  <<<parent >>>| lvl2 |<<< child >>>| lvl3 |<<< predicted for parent >>>  10020  <<<parent >>>| lvl2 |<<< child >>>| lvl3 |<<< predicted for parent >>>  10018  <<<parent >>>| lvl2 |<<< child >>>| lvl3 |<<< predicted for parent >>>  10003  <<<parent >>>| lvl2 |<<< child >>>| lvl3 |<<< predicted for parent >>>  10014  <<<parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10113  <<<parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10044  <<<parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10118  <<<parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10023  <<<parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10191  <<<parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10110  <<<parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for

D:\Programming\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10219  <<<parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10021  <<<parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10073  <<<parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10095  <<<parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10185  <<<

D:\Programming\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10166  <<<parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10184  <<<parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10091  <<<parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10070  <<<parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10049  <<<parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10221  <<<parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10058  <<<parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10180  <<<parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10026  <<<parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10084  <<<parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10074  <<<parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10141  <<<parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted

...

parent >>>| lvl5 |<<< child >>>| lvl6 |<<< predicted for parent >>>  11281  <<<parent >>>| lvl5 |<<< child >>>| lvl6 |<<< predicted for parent >>>  2815  <<<

D:\Programming\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


parent >>>| lvl5 |<<< child >>>| lvl6 |<<< predicted for parent >>>  2812  <<<parent >>>| lvl5 |<<< child >>>| lvl6 |<<< predicted for parent >>>  13073  <<<parent >>>| lvl5 |<<< child >>>| lvl6 |<<< predicted for parent >>>  2821  <<<parent >>>| lvl5 |<<< child >>>| lvl6 |<<< predicted for parent >>>  13806  <<<parent >>>| lvl5 |<<< child >>>| lvl6 |<<< predicted for parent >>>  2818  <<<parent >>>| lvl5 |<<< child >>>| lvl6 |<<< predicted for parent >>>  2814  <<<parent >>>| lvl5 |<<< child >>>| lvl6 |<<< predicted for parent >>>  2817  <<<>>> DONE WITH WHOLE DF <<<
Wall time: 4min 42s


D:\Programming\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


,i,predicted_lvl,parent_lvl,f1_holdout,length,wgt_in_lvl,f1_lvl_weighted,cval_f1_mean,cval_f1_std,cval_mean_weighted
0,1,lvl2,lvl1,0.988630,283452,1.000000,0.988630,0.986561,0.000448,0.986561
1,10012,lvl3,lvl2,0.909150,51834,0.182867,0.166253,0.899137,0.002138,0.164422
2,10020,lvl3,lvl2,0.902690,48816,0.172220,0.155461,0.845096,0.008036,0.145542
3,10018,lvl3,lvl2,0.923495,72116,0.254421,0.234956,0.916324,0.000303,0.233132
4,10003,lvl3,lvl2,0.775658,55293,0.195070,0.151308,0.759389,0.007847,0.148134
...,...,...,...,...,...,...,...,...,...,...
240,2821,lvl6,lvl5,0.927273,230,0.022904,0.021238,0.757989,0.130751,0.017361
241,13806,lvl6,lvl5,0.589524,124,0.012348,0.007280,0.796673,0.055083,0.009837
242,2818,lvl6,lvl5,0.935714,157,0.015634,0.014629,0.481604,0.083051,0.007530
243,2814,lvl6,lvl5,1.000000,4,0.000398,0.000398,1.000000,0.000000,0.000398


Посчитаем взвешенные результаты на каждом уровне. В целом, модель неплохо отрабатывает на уровнях 2, 4, 5.<br> Есть узкое место на уровне 3 - возможно, помог бы тьюнинг параметров или выбор другого классификатора.

In [21]:
overal_res = df_res.groupby('predicted_lvl')[['f1_lvl_weighted', 
                                                             'cval_mean_weighted', 'length']].sum().reset_index()
overal_res['lvl_weight'] = overal_res['length']/overal_res['length'].max()
overal_res

,predicted_lvl,f1_lvl_weighted,cval_mean_weighted,length,lvl_weight
0,lvl2,0.988630,0.986561,283452,1.000000
1,lvl3,0.875549,0.855672,283452,1.000000
2,lvl4,0.917422,0.905805,283301,0.999467
3,lvl5,0.895108,0.858684,240383,0.848055
4,lvl6,0.801358,0.691645,10042,0.035428


Посмотрим, где модель плохо справляется. Гипотетически, можно было бы для самых слабых моделей заменять классификаторы/проводить тьюнинг параметров внутри цикла.

In [22]:
df_res[df_res['length'] > 1000].sort_values(by='cval_f1_mean', ascending=True).head(10)

,i,predicted_lvl,parent_lvl,f1_holdout,length,wgt_in_lvl,f1_lvl_weighted,cval_f1_mean,cval_f1_std,cval_mean_weighted
147,2808,lvl5,lvl4,0.499010,1012,0.004210,0.002101,0.499010,0.000000,0.002101
236,11281,lvl6,lvl5,0.757844,3377,0.336288,0.254854,0.607507,0.088686,0.204297
138,10597,lvl5,lvl4,0.694716,2558,0.010641,0.007393,0.618411,0.041697,0.006581
76,10908,lvl5,lvl4,0.687785,1649,0.006860,0.004718,0.628506,0.112887,0.004311
93,11390,lvl5,lvl4,0.777965,1075,0.004472,0.003479,0.644338,0.025565,0.002881
17,10144,lvl4,lvl3,0.824192,3773,0.013318,0.010977,0.673031,0.164104,0.008963
66,10313,lvl5,lvl4,0.832450,3414,0.014202,0.011823,0.693169,0.142004,0.009845
105,10143,lvl5,lvl4,0.857020,1940,0.008070,0.006917,0.704733,0.041264,0.005688
80,10992,lvl5,lvl4,0.809594,2368,0.009851,0.007975,0.723908,0.023911,0.007131
19,10129,lvl4,lvl3,0.910648,1852,0.006537,0.005953,0.725333,0.047610,0.004742


## Предсказание

Воспользуемся алгоритмом выше для классификации тестовой выборки.

Сделаем препроцессинг.

In [23]:
%%time
preproc_df(df_test)
df_test.head(3)

>>> Merged columns
>>> Done with title
>>> Done with name_value_characteristics
Wall time: 2min 35s


,id,title,short_description,name_value_characteristics,rating,feedback_quantity
0,1070974,браслет натуральный камень lotus 0dot0,0,0,0.000000,0
1,450413,fusion life шампунь сухой окрасить волос личит...,0,0,4.333333,6
2,126857,микрофон пк jack 3dot5мма всенаправленный унив...,"универсальный 3,5 мм микрофон запишет ваш звук",0,3.708333,24


Вспомним, что у нас все товары в тренировочном сете попадали в категорию 1. Пропишем это и для тестового сета.

In [24]:
df_predictions_test = pd.DataFrame({'id': df_test['id'],
                                    'title': df_test['title'],
                                    'y_pred': [1 for x in df_test['title']],
                                    'lvl_p': ['lvl0' for x in df_test['title']],
                                    'lvl_d': ['lvl1' for x in df_test['title']] 
                                   })
df_predictions_test.head(3)

,id,title,y_pred,lvl_p,lvl_d
0,1070974,браслет натуральный камень lotus 0dot0,1,lvl0,lvl1
1,450413,fusion life шампунь сухой окрасить волос личит...,1,lvl0,lvl1
2,126857,микрофон пк jack 3dot5мма всенаправленный унив...,1,lvl0,lvl1


Предскажем классы для меньшего ранга по тестовой выборке, двигаясь от lvl1 вниз.

In [25]:
%%time
df_test['title']

# задаем листы с предками-потомками, по которым будем итерировать модели
p_c = ['lvl1', 'lvl2', 'lvl3', 'lvl4', 'lvl5'] 
d_c = ['lvl2', 'lvl3', 'lvl4', 'lvl5', 'lvl6']

# основной цикл
for p, d in zip(p_c, d_c):
    
    # движемся от начала древа вниз. фильтруем по классу-предку, классифицируем по подклассам на уровень ниже
    for i in df_train[(df_train[p] != 0) & (df_train[d] != 0)][p].unique():
        
        # выбираем Х и у из тренировочной выборки с таким же предком, как и Х в тестовой выборки 
        # (для тестовой класс предка предсказан или задан = 1 для lvl1)
        X = df_train[(df_train[p] == i)]['title'].astype(str)
        y = df_train[(df_train[p] == i)][d]
        X_test = df_predictions_test[(df_predictions_test['lvl_d'] == p) 
                                     & (df_predictions_test['y_pred'] == i)]['title'].astype(str)
        X_test_id = df_predictions_test[(df_predictions_test['lvl_d'] == p) 
                                     & (df_predictions_test['y_pred'] == i)]['id']
        
        # если в предках тестовой выборки нет такой категории -> вернемся в начало цикла
        if len(X_test) == 0:
            continue
        else:
            
            # логгирование - начало итерации
            # print('working on >>> ', i)
            
            # были категории-родители с только одним потомком - для таких присваиваем руками класс единственного потомка
            if len(y.unique()) == 1:
                y_pred = y.iloc[0]
                df_predictions_test_small = pd.DataFrame({'id': X_test_id,
                                                          'title': X_test,
                                                          'y_pred': [y_pred for x in X_test],
                                                          'lvl_p': [p for x in X_test],
                                                          'lvl_d': [d for x in X_test]}
                                                        )
                
            # создание модели и предсказание в стандартных ситуациях                
            else:
                
                # трансформация переменных
                Z = tfidf.fit_transform(X)
                Z_test = tfidf.transform(X_test)
                y = label_encoder.fit_transform(y)
                
                # фиттинг 
                clf.fit(Z, y)
                y_pred = clf.predict(Z_test)
                
                # запись результатов в мини-датафрейм
                df_predictions_test_small = pd.DataFrame({'id': X_test_id,
                                                          'title': X_test,
                                                          'y_pred': label_encoder.inverse_transform(y_pred),
                                                          'lvl_p': [p for x in X_test],
                                                          'lvl_d': [d for x in X_test]}
                                                        )

        # присоединение результатов мини-датафрейма к основному датафрейму
        df_predictions_test = pd.concat([df_predictions_test, df_predictions_test_small], axis=0, ignore_index=True)
        
        # логгирование - конец итерации
        print('parent >>>|', p, '|<<< child >>>|', d, '|<<< predicted for parent >>> ', i, ' <<<')
print('>>> DONE WITH WHOLE DF <<<')
df_predictions_test

parent >>>| lvl1 |<<< child >>>| lvl2 |<<< predicted for parent >>>  1  <<<
parent >>>| lvl2 |<<< child >>>| lvl3 |<<< predicted for parent >>>  10012  <<<
parent >>>| lvl2 |<<< child >>>| lvl3 |<<< predicted for parent >>>  10020  <<<
parent >>>| lvl2 |<<< child >>>| lvl3 |<<< predicted for parent >>>  10018  <<<
parent >>>| lvl2 |<<< child >>>| lvl3 |<<< predicted for parent >>>  10003  <<<
parent >>>| lvl2 |<<< child >>>| lvl3 |<<< predicted for parent >>>  10014  <<<
parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10113  <<<
parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10044  <<<
parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10118  <<<
parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10023  <<<
parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10191  <<<
parent >>>| lvl3 |<<< child >>>| lvl4 |<<< predicted for parent >>>  10110  <<<
parent >>>| lvl3 |<<< child >>>| lvl4 |<<< p

parent >>>| lvl4 |<<< child >>>| lvl5 |<<< predicted for parent >>>  10890  <<<
parent >>>| lvl4 |<<< child >>>| lvl5 |<<< predicted for parent >>>  10143  <<<
parent >>>| lvl4 |<<< child >>>| lvl5 |<<< predicted for parent >>>  10476  <<<
parent >>>| lvl4 |<<< child >>>| lvl5 |<<< predicted for parent >>>  11285  <<<
parent >>>| lvl4 |<<< child >>>| lvl5 |<<< predicted for parent >>>  10559  <<<
parent >>>| lvl4 |<<< child >>>| lvl5 |<<< predicted for parent >>>  10858  <<<
parent >>>| lvl4 |<<< child >>>| lvl5 |<<< predicted for parent >>>  10260  <<<
parent >>>| lvl4 |<<< child >>>| lvl5 |<<< predicted for parent >>>  11254  <<<
parent >>>| lvl4 |<<< child >>>| lvl5 |<<< predicted for parent >>>  10225  <<<
parent >>>| lvl4 |<<< child >>>| lvl5 |<<< predicted for parent >>>  10081  <<<
parent >>>| lvl4 |<<< child >>>| lvl5 |<<< predicted for parent >>>  10404  <<<
parent >>>| lvl4 |<<< child >>>| lvl5 |<<< predicted for parent >>>  10634  <<<
parent >>>| lvl4 |<<< child >>>| lvl5 |<

parent >>>| lvl4 |<<< child >>>| lvl5 |<<< predicted for parent >>>  10922  <<<
parent >>>| lvl4 |<<< child >>>| lvl5 |<<< predicted for parent >>>  10418  <<<
parent >>>| lvl4 |<<< child >>>| lvl5 |<<< predicted for parent >>>  10075  <<<
parent >>>| lvl4 |<<< child >>>| lvl5 |<<< predicted for parent >>>  10879  <<<
parent >>>| lvl4 |<<< child >>>| lvl5 |<<< predicted for parent >>>  10805  <<<
parent >>>| lvl4 |<<< child >>>| lvl5 |<<< predicted for parent >>>  2811  <<<
parent >>>| lvl4 |<<< child >>>| lvl5 |<<< predicted for parent >>>  10950  <<<
parent >>>| lvl4 |<<< child >>>| lvl5 |<<< predicted for parent >>>  10646  <<<
parent >>>| lvl4 |<<< child >>>| lvl5 |<<< predicted for parent >>>  10915  <<<
parent >>>| lvl4 |<<< child >>>| lvl5 |<<< predicted for parent >>>  11140  <<<
parent >>>| lvl4 |<<< child >>>| lvl5 |<<< predicted for parent >>>  11168  <<<
parent >>>| lvl4 |<<< child >>>| lvl5 |<<< predicted for parent >>>  10203  <<<
parent >>>| lvl4 |<<< child >>>| lvl5 |<<

,id,title,y_pred,lvl_p,lvl_d
0,1070974,браслет натуральный камень lotus 0dot0,1,lvl0,lvl1
1,450413,fusion life шампунь сухой окрасить волос личит...,1,lvl0,lvl1
2,126857,микрофон пк jack 3dot5мма всенаправленный унив...,1,lvl0,lvl1
3,1577569,серьга гвоздик сердце серьга гвоздик сердце 0,1,lvl0,lvl1
4,869328,чёрно красный стильный брошь тюльпан акрил бро...,1,lvl0,lvl1
...,...,...,...,...,...
346219,1218054,костюм футер mini тёплый костюм мальчик девочк...,2851,lvl5,lvl6
346220,1395972,футболка детский оранжевый футболка оранжевый 0,2855,lvl5,lvl6
346221,1115813,детский спортивный костюм детский спортивный к...,2851,lvl5,lvl6
346222,701098,костюм спортивный девочка lokki cute girl rule...,2851,lvl5,lvl6


Удалим дубликаты.

In [26]:
df_predictions_test_compact = df_predictions_test[['id', 'title', 
                                                   'y_pred', 'lvl_d']].drop_duplicates(subset='id', keep='last')

Проверим, что ничего не потерялось по дороге.

In [27]:
print(f'Длина исходного неразмеченного фрейма = {len(df_test)}')
print(f'Длина фрейма с предсказаниями = {len(df_predictions_test_compact)}')

Длина исходного неразмеченного фрейма = 70864
Длина фрейма с предсказаниями = 70864


Подсоединим названия категорий.

In [28]:
df_predictions_test_compact = df_predictions_test_compact.merge(
    cat_merged[['deep_lvl', 'deep_id']], how='left', left_on='y_pred', right_on='deep_id')
df_predictions_test_compact = df_predictions_test_compact.merge(
    df_categories[['title', 'id']], how='left', left_on='y_pred', right_on='id')
df_predictions_test_compact.head(3)

,id_x,title_x,y_pred,lvl_d,deep_lvl,deep_id,title_y,id_y
0,1311421,мусульманский платье 3dot0 0,12084,lvl3,lvl3,12084,Религиозная одежда,12084
1,978675,чётки змеевик sautoir чётки натуральный камень 0,12084,lvl3,lvl3,12084,Религиозная одежда,12084
2,1499127,коврик намаз молитвенный коврик совершение еже...,12084,lvl3,lvl3,12084,Религиозная одежда,12084


Запишем предсказанные категории в паркет.

In [29]:
submission = df_predictions_test_compact[['id_x', 'y_pred']]
submission.columns = ['id', 'predicted_category_id']
submission.head(3)

,id,predicted_category_id
0,1311421,12084
1,978675,12084
2,1499127,12084


In [30]:
submission.to_parquet('result.parquet', index=False)

In [31]:
df_res = pd.read_parquet('result.parquet')
df_res.head(3)

,id,predicted_category_id
0,1311421,12084
1,978675,12084
2,1499127,12084
